In [ ]:
set.seed(1)
S <- 10000 #number of samples
n <- 50 #number of observations per sample

b0 <- 2
b1 <- 3
b2 <- 0.9

e <- replicate(S,rnorm(n)) #draw S samples

x <- 1:n

y<-matrix(,n,S)
y[1,]<-b0 + b1*x + e[1,] # generate the first value
for (t in 2:n){ #loop over all observtions
    y[t,]<- b0 + b1*x[t] +b2*y[t-1,]+ e[t,] # generate succesive value for y
    }

In [ ]:
lm.fit=lm(y[2:n,1]~x[2:n]+y[1:(n-1),1])
summary(lm.fit)


Call:
lm(formula = y[2:n, 1] ~ x[2:n] + y[1:(n - 1), 1])

Residuals:
     Min       1Q   Median       3Q      Max 
-2.32920 -0.45574 -0.00961  0.60252  1.37291 

Coefficients:
                Estimate Std. Error t value Pr(>|t|)    
(Intercept)     2.373099   0.633518   3.746    5e-04 ***
x[2:n]          2.961767   0.108169  27.381   <2e-16 ***
y[1:(n - 1), 1] 0.901348   0.004069 221.518   <2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 0.8494 on 46 degrees of freedom
Multiple R-squared:      1,	Adjusted R-squared:      1 
F-statistic: 4.92e+06 on 2 and 46 DF,  p-value: < 2.2e-16


In [ ]:
bh=matrix(,3,S) #initialize matrix to store estimates of beta
reject=rep(0,S,1)
for (s in 1:S){
    lm.fs.fit=lm(y[2:n,s]~x[2:n]+y[1:(n-1),s]) # estimate the linear model on the sample s.
    bh[1:3,s] <- coef(lm.fs.fit)   #store the estimates 
    reject[s] = (abs((summary(lm.fs.fit)$coef[3,1]-0.9) / summary(lm.fs.fit)$coef[3,2] )>qt(.975,n-4)) 
}

sd(bh[1,])
sd(bh[2,])
sd(bh[3,])

[1] 0.4980181

[1] 0.09432588

[1] 0.003671368

In [ ]:
mean(reject)

[1] 0.0509

### Now run bootstrap

In [ ]:
residuals <-lm.fit$residuals

B <- 9999 #number of samples
ts=matrix(,3,B) #initialize matrix to store test statistics from linear regression

for (b in 1:B){
    e.bs <-sample(residuals, replace=TRUE) #for each loop, generate a new sample of residuals WITH REPLACEMENT
    
    y.bs<-rep(0,n,S)
    y.bs[1]<-b0 + b1*x[1] + e.bs[1] # generate the first value (i.e. assume y(0)=0)
    for (t in 2:n){ #loop over all observtions
        y.bs[t]<- b0 + b1*x[t] + 0.9*y.bs[t-1] + e.bs[t] # generate succesive value for y under null hypothesis of beta_2=0.9!
    }
    
    lm.bs.fit=lm(y.bs[2:n]~x[2:n]+y.bs[1:(n-1)]) # estimate the linear model on the bootstrapped sample.
    ts[1:3,b] <- summary(lm.bs.fit)$coefficient[,3]  #store the t-statistics from this sample 
}

In [ ]:
quantile(ts[3,],0.975)
quantile(ts[3,],0.025)

97.5% 
399.6003

2.5% 
297.2587

In [ ]:
qt(0.975,n-1) #the 97.5% from the t-distribution
qt(0.025,n-1)

[1] 1.984217

[1] -1.984217

In [ ]:
tstat <- summary(lm.fit)$coefficient[3,3] #the original t-stat

mean(abs(ts[3,])> tstat) #what proportion of bootstrap test statistics lie above the original test statistic?

[1] 0.59986